In [1]:
import dotenv
dotenv.load_dotenv()
from pinecone import Pinecone, ServerlessSpec
# from openai import OpenAI
import os




/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import requests
import json

response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {'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'}",
    # "HTTP-Referer": f"{YOUR_SITE_URL}", # Optional, for including your app on openrouter.ai rankings.
    # "X-Title": f"{YOUR_APP_NAME}", # Optional. Shows in rankings on openrouter.ai.
  },
  data=json.dumps({
    "model": "meta-llama/llama-3.1-8b-instruct:free", # Optional
    "messages": [
      { "role": "user", "content": "What is the meaning of life?" }
    ]
    
  })
)

In [5]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


In [16]:
import json
data = json.load(open("reviews.json"))
data

[{'professor': 'Dr. Smith',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Engaging lectures, but tough grader.'},
 {'professor': 'Prof. Johnson',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Excellent at explaining complex concepts.'},
 {'professor': 'Dr. Lee',
  'subject': 'Chemistry',
  'stars': 3,
  'review': 'Knowledgeable, but moves too fast.'},
 {'professor': 'Prof. Garcia',
  'subject': 'History',
  'stars': 4,
  'review': 'Passionate about the subject.'},
 {'professor': 'Dr. Patel',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Very helpful during office hours.'},
 {'professor': 'Prof. Brown',
  'subject': 'English Literature',
  'stars': 4,
  'review': 'Insightful discussions in class.'},
 {'professor': 'Dr. Wilson',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Difficult exams, but fair.'},
 {'professor': 'Prof. Taylor',
  'subject': 'Psychology',
  'stars': 5,
  'review': 'Makes the material relatable.'},
 {'professor': 'Dr. Anderson',
  'su

In [19]:
for i in data:
    print(i["review"]) 

Engaging lectures, but tough grader.
Excellent at explaining complex concepts.
Knowledgeable, but moves too fast.
Passionate about the subject.
Very helpful during office hours.
Insightful discussions in class.
Difficult exams, but fair.
Makes the material relatable.
Clear explanations of complex theories.
Interesting topics, but heavy workload.
Passionate and knowledgeable.
Encourages critical thinking.
Field trips are great, lectures less so.
Challenging but rewarding classes.
Engaging storyteller and educator.
Knows the subject well, but strict.
Inspiring and practical approach.
Patient and thorough explanations.
Interesting material, but disorganized.
Thought-provoking discussions.


In [21]:
processed_data = []
client = OpenAI()
data = [data[0]]

for review in data:
    response = client.embeddings.create(
        input=review["review"], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}